In [34]:
import csv
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

In [35]:
# Funkce pro převod délky z sekund na formát hh:mm:ss
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Funkce na získání délky zvukového souboru v sekundách pomocí librosa
def get_audio_length(file_path):
    y, sr = librosa.load(file_path, sr=None)  # Načtení zvuku s původní vzorkovací frekvencí
    duration = librosa.get_duration(y=y, sr=sr)
    return duration

# Funkce pro filtrování řádků podle délky zvukového souboru a výpočet celkových délek
def filter_audio_files(input_csv, audio_directory, max_length, output_csv):
    total_original_length = 0  # Celková délka všech původních zvukových souborů
    total_output_length = 0  # Celková délka zvukových souborů ve výstupu
    input_files = []  # Seznam vstupních souborů s délkou
    output_files = []  # Seznam výstupních souborů s délkou

    with open(input_csv, encoding='utf-8') as infile, open(output_csv, 'w', encoding='utf-8') as outfile:
        reader = csv.reader(infile, delimiter='|')
        writer = csv.writer(outfile, delimiter='|', lineterminator='\n')
        
        # Zapsání hlavičky, pokud existuje
        headers = next(reader, None)
        if headers:
            writer.writerow(headers)

        for row in reader:
            file_name = row[0]  # Název souboru je první položka v řádce

            # Přidání přípony .wav, pokud tam není
            if not file_name.lower().endswith('.wav'):
                file_name += '.wav'
            
            audio_file_path = os.path.join(audio_directory, file_name)
            
            # Pokud zvukový soubor existuje, zjisti jeho délku
            if os.path.isfile(audio_file_path):
                length = get_audio_length(audio_file_path)
                total_original_length += length  # Přičti k celkové délce původních souborů
                input_files.append((file_name, length))  # Přidej soubor a jeho délku do seznamu

                # Pokud délka je menší než zadaná hodnota, zapiš řádek do výstupního souboru
                if length < max_length:
                    writer.writerow(row)
                    output_files.append((file_name, length))  # Přidej soubor do seznamu výstupních souborů
                    total_output_length += length  # Přičti k celkové délce výstupních souborů
    return total_original_length, total_output_length, input_files, output_files

In [36]:
# Nastavení
input_csv = '../Data/DailyTalk0.en/train.phn.csv'  # cesta k vstupnímu CSV souboru
audio_directory = '../Data/DailyTalk0.en/wavs'  # cesta k adresáři se zvukovými soubory
max_length = 5  # maximální povolená délka zvukového souboru v sekundách
output_csv = f'../Data/DailyTalk0.en/train{max_length}s.phn.csv'  # cesta k výstupnímu CSV souboru

total_original_length, total_output_length, input_files, output_files = filter_audio_files(
    input_csv,
    audio_directory,
    max_length,
    output_csv
)

In [ ]:
# Výpočty průměrné délky a mediánu
input_lengths = [length for _, length in input_files]
output_lengths = [length for _, length in output_files]

avg_input_length = np.mean(input_lengths) if input_lengths else 0
median_input_length = np.median(input_lengths) if input_lengths else 0

avg_output_length = np.mean(output_lengths) if output_lengths else 0
median_output_length = np.median(output_lengths) if output_lengths else 0

print(f"Celkový počet vstupních audiosouborů: {len(input_files)}")
print(f"Celková délka vstupních audiosouborů: {format_time(total_original_length)}")
print(f"Průměrná délka vstupních audiosouborů: {avg_input_length:.2f}")
print(f"Medián délky vstupních audiosouborů: {median_input_length:.2f}")

print(f"\nCelkový počet výstupních audiosouborů: {len(output_files)}")
print(f"Celková délka výstupních audiosouborů: {format_time(total_output_length)}")
print(f"Průměrná délka výstupních audiosouborů: {avg_output_length:.2f}")
print(f"Medián délky výstupních audiosouborů: {median_output_length:.2f}")

In [ ]:
# Vytvoření histogramu délek výstupních souborů
if input_lengths:
    plt.hist(input_lengths, bins=100, edgecolor='black')
    plt.title('Histogram délek vstupních audio souborů')
    plt.xlabel('Délka (v sekundách)')
    plt.ylabel('Počet souborů')
    plt.show()

In [ ]:
# Vytvoření histogramu délek výstupních souborů
if output_lengths:
    plt.hist(output_lengths, bins=100, edgecolor='black')
    plt.title('Histogram délek výstupních audio souborů')
    plt.xlabel('Délka (v sekundách)')
    plt.ylabel('Počet souborů')
    plt.show()